# What is inside a smart contract?

Every smart contract has a set of methods that can be exposed to outside users, or kept internally for the operations of the smart contract. Each smart contract also has a data namespace on which it can read and write data too. Smart contracts can read data from other smart contracts, but they cannot write to other smart contracts.

In [10]:
def basic_contract():
    # data section
    owner = Variable()
    balances = Hash(default_value=0)
    
    @construct
    def seed():
        owner.set('stuart')
        balances['stuart'] = 1000000

    @export
    def mint(amount, to):
        assert_is_owner()
        
        balances[to] += amount
        
    def assert_is_owner():
        assert ctx.caller == owner.get(), 'You are not the owner! {} is!'.format(owner.get())
        
    @export
    def send(amount, to):
        assert balances[ctx.caller] >= amount, 'You do not have enough to send! {} is less than {}'.format(
            balances[ctx.caller], amount
        )
        
        balances[ctx.caller] -= amount
        balances[to] += amount

## Functions
This looks like a lot to chew over, but let's go over it piece by piece. There are 3 types of functions:

* @construct
    
    This function is run once on contract submission. It 'constructs' the initial state of the smart contract. Use this for setting initial variables. You can only have one @construct function, but you can call it whatever you want.
    
    
* @export
    
    These functions are callable by outside users and other smart contracts. Therefore, you have to pay extra attention to the code you write in them. If @export functions do things that you do not want every user to do, you should put them in a private function, or add access control features.
    
    
* everything else
    
    Any other function without a decorator is a private method that can only be called by the contract itself. These functions are good for complex contracts that need to abstract and reuse some logic over and over again. It also allows you to set apart the pieces of the smart contract logic between core controller code, and external interaction functions.

In [14]:
from contracting.client import ContractingClient
client = ContractingClient(signer='stuart')

In [15]:
client.submit(basic_contract)
contract = client.get_contract('basic_contract')

Because @constructor sets `stuart`'s balance to 1,000,000, we can access the variable directly from our contract object to check.

In [22]:
contract.balances['stuart']

1000000

Because `mint` is an @export function, we can call it from the outside. Remember, the `client` was initialized with the `signer` equal to `stuart`.

In [21]:
contract.mint(amount=10000, to='raghu')
contract.balances['raghu']

20000

If we override the signer keyword to someone who is `not_stuart`, the function will fail because of the assertation. The internal function is called here.

In [18]:
try:
    contract.mint(amount=500, to='raghu', signer='not_stuart')
except AssertionError as e:
    print(e)

You are not the owner! stuart is!


However, if you try to access the function normally (and remember to prepend the `__` before it), you can see that the function is not available to us to call.

In [28]:
try:
    contract.__assert_is_owner()
except Exception as e:
    print(e)

module 'basic_contract' has no attribute '__assert_is_owner'


## Data
You will also notice a `Variable` object and `Hash` object initialized in the beginning of the smart contract. These are data variables that are used to store data on the smart contract. You cannot modify these variables directly. Instead, the @export functions you write determine how data is modified in these variables. This allows you to create extremely robust and secure data models for your application.

There are also `ForeignVariable` and `ForeignHash` objects. These are 'read-only' variables that allow your smart contract to import the namespace of another smart contract for internal use, but prevents writing to them. For example, your function might only pass given certain conditions on another smart contract that it is watching. Foreign varibles would be used in this case.

The Contracting client exposes these variables into Python objects and *does* allow modification. This is so that the developer can test various situations more easily. It does not reflect how users will be able to modify data in a smart contract namespace.

In [37]:
def data_contract():
    basic_contract_owner = ForeignVariable(foreign_contract='basic_contract', foreign_name='owner')
    
    variable = Variable()
    hash_ = Hash()
    
    # Demonstration of returning a value in another smart contract's namespace
    @export
    def whos_the_owner():
        return basic_contract_owner.get()
    
    @export
    def set_var(x):
        variable.set(x)
        
    @export
    def set_hash(k, v):
        hash_[k] = v

In [38]:
client.submit(data_contract)
d_contract = client.get_contract('data_contract')

#### Read the Foreign Variable

In [39]:
d_contract.whos_the_owner()

'stuart'

#### Set the Variable via Smart Contract and via Client

In [44]:
d_contract.set_var(x=1000)
d_contract.variable.get()

1000

In [45]:
d_contract.set_var(x=123)
d_contract.variable.get()

123

In [46]:
# Overriding our method to modify the variable directly
d_contract.variable.set(555)
d_contract.variable.get()

555

#### Set the Hash via Smart Contract and via Client

In [52]:
d_contract.set_hash(k='hello', v='world')
d_contract.hash_['hello']

'world'

In [53]:
d_contract.hash_['hello'] = 'there'
d_contract.hash_['hello']

'there'

### Behavior Note

Variables and hashes only take up space in the database if they are initialized. Therefore, you cannot access a hash that has no values. Because you cannot access this hash (or variable), you cannot manually set values on it either. Make sure that you always either @construct your variables, or provide methods in your smart contract to interact with them.

In [56]:
def variable_behavior():
    invisible_variable = Variable()
    
    @export
    def do_nothing():
        return 0

In [57]:
client.submit(variable_behavior)
v_contract = client.get_contract('variable_behavior')

In [59]:
try:
    v_contract.invisible_variable
except AttributeError as e:
    print(e)

'AbstractContract' object has no attribute 'invisible_variable'
